In [6]:
import aiogram
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import InlineKeyboardButton, InlineKeyboardMarkup, ReplyKeyboardMarkup

bot = Bot(token='5918472294:AAGYqlSwmdwbFvnTroam00EyIRfL4tnINDE')
dp = Dispatcher(bot)

async def mood_tracker(message):
    keyboard = InlineKeyboardMarkup()
    happy_button = InlineKeyboardButton('Happy', callback_data='happy')
    meh_button = InlineKeyboardButton('Meh', callback_data='meh')
    sad_button = InlineKeyboardButton('Sad', callback_data='sad')
    keyboard.add(happy_button, meh_button, sad_button)
    text = "What's your mood today?"
    await bot.send_message(chat_id=message.from_user.id, text=text, reply_markup=keyboard)
    
async def weather_tracker(message):
    keyboard = ReplyKeyboardMarkup()
    button = KeyboardButton(text="Send my location", request_location=True)
    keyboard.add(button)
    text = "Please send your location to get the current weather"
    await bot.send_message(chat_id=message.from_user.id, text=text, reply_markup=keyboard)
    
async def get_moon_phase(date):
    url = f'http://api.usno.navy.mil/moon/phase?date={date}&nump={1}'
    response = requests.get(url)
    data = response.json()
    phase = data['phasedata'][0]['phase']
    return phase

async def moon_phase_tracker(message):
    date = datetime.datetime.now().strftime("%Y-%m-%d")
    phase = get_moon_phase(date)
    text = f"The current moon phase is {phase}."
    await bot.send_message(chat_id=message.from_user.id, text=text)

@dp.message_handler(content_types=['location'])
async def process_location(message: types.Message):
    lat = message.location.latitude
    lon = message.location.longitude
    url = f'http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={"45bd10a5036ab9086034fac86c62ac73"}'
    response = requests.get(url)
    data = response.json()
    weather = data['weather'][0]['main']
    mood = get_mood(weather)
    text = f"The current weather in your location is {weather}. I'm feeling {mood}."
    await bot.send_message(chat_id=message.from_user.id, text=text)
    
@dp.callback_query_handler(lambda c: c.data == 'check_weather')
async def process_callback_weather_tracker(callback_query: types.CallbackQuery):
    await weather_tracker(callback_query.message)
    
@dp.callback_query_handler(lambda c: c.data in ['happy', 'meh', 'sad'])
async def process_callback_mood(callback_query: aiogram.types.CallbackQuery):
    mood = callback_query.data
    await bot.send_message(chat_id=callback_query.from_user.id, text=f"Your mood is {mood}")
    
@bot.message_handler(commands=['start'])
async def start_command(message: aiogram.types.Message):
    keyboard = InlineKeyboardMarkup()
    weather_button = InlineKeyboardButton('Weather', callback_data='weather')
    mood_button = InlineKeyboardButton('Mood', callback_data='mood')
    moon_button = InlineKeyboardButton('Moon', callback_data='moon')
    keyboard.add(weather_button, mood_button, moon_button)
    text = "Welcome to the Weather, Mood, and Moon Phase Tracker Bot! What would you like to track?"
    await bot.send_message(chat_id=message.from_user.id, text=text, reply_markup=keyboard)

@bot.callback_query_handler(lambda c: c.data == 'weather')
async def process_callback_weather(callback_query: aiogram.types.CallbackQuery):
    await weather_tracker(callback_query.message)

@bot.callback_query_handler(lambda c: c.data == 'mood')
async def process_callback_mood(callback_query: aiogram.types.CallbackQuery):
    await mood_tracker(callback_query.message)

@bot.callback_query_handler(lambda c: c.data == 'moon')
async def process_callback_moon(callback_query: aiogram.types.CallbackQuery):
    await moon_phase_tracker(callback_query.message)


In [7]:
# this is the last line
executor.start_polling(dp)

RuntimeError: This event loop is already running